In [79]:
#將id group  做成字典
#step 1
def put_id_group_in_dic(id_group_file_path):
    with open(id_group_file_path, 'r') as f:
        groupnum= f.read()
    groupnum.split('\n')
    group_dic={}
    for one_id_group in groupnum.split('\n'):
        one_patent_id=one_id_group.split(',')[0][1:]
        one_patent_groupnum=one_id_group.split(',')[1][0:-1]
        group_dic[one_patent_id]=one_patent_groupnum
    print "step 1 finish"
    return group_dic

In [80]:
#將該ID切出的字併入上面的字典
#step 2
def merge_keywords(id_keywords_file_path, group_dic):
    word_list=[]
    with open(id_keywords_file_path, 'r') as f:
        all_patent= f.read()
    f.close()
    for one_patent in all_patent.split('\n'):
        one_patent_id=one_patent.split(",")[0].strip()
        one_patent_keyword=one_patent.split(",")[1].strip()
        for one_word in one_patent_keyword.split(" "):
            word_list.append(one_word)
        new_value =list(group_dic[one_patent_id]) + word_list
        group_dic[one_patent_id]=new_value   
    print "step 2 finish"
    return group_dic

In [81]:
#合併寫法160篇  大約7秒
#step 3
def source_to_big_dic(source_dic):
    big_dic={}
    count=0
    for _id in source_dic:  #一筆一筆抓_id
        count+=1
        print count
        I_got_this = 0  #清除抓到旗標
    #=======================================以下為判斷是否已有該群組之程式碼=====================================
        for group_num in big_dic:    #將大字典中的當前群組抓出來
            if source_dic[_id][0]==group_num:    #比對該群組是否等於該筆id所屬的群組
                I_got_this = 1               #有抓到就豎起抓到旗標

    #=====================================以下為入字庫程式碼==============================================
                big_dic[source_dic[_id][0]][source_dic[_id][0]+"_id"].append(_id)
                for one_words in source_dic[_id]:                #開始把該筆_ID內的關鍵字取出
                    if one_words not in big_dic[group_num]:   #判斷是否已在該群組中出現   沒有就建新字
                        big_dic[group_num][one_words] = 1      #建出目前判斷的字
                    else: 
                        big_dic[group_num][one_words]=big_dic[group_num][one_words]+1  #更新該字詞之數量

    #====================================以下違建立新群組程式碼=====並建立"0~n"_id之KEY========================     
        if I_got_this == 0:         #判斷抓到旗標，若沒豎起表示沒抓到
            big_dic[source_dic[_id][0]]={}     #在big_dic中建立新的key (新群組)
            big_dic[source_dic[_id][0]][source_dic[_id][0]+"_id"]=[]    #在新建的群組中在建立一個"0_id"的list
            group_num=source_dic[_id][0]       #將新群組之號碼設成要入之字庫

    #=====================================以下為入字庫程式碼=============================================
            big_dic[source_dic[_id][0]][source_dic[_id][0]+"_id"].append(_id)  
            for one_words in source_dic[_id]:                #開始把該筆_ID內的關鍵字取出
                if one_words not in big_dic[group_num]:   #判斷是否已在該群組中出現   沒有就建新字
                    big_dic[group_num][one_words] = 1      #建出目前判斷的字
                else: 
                    big_dic[group_num][one_words]=big_dic[group_num][one_words]+1  #更新該字詞之數量
    print "step 3 finish","="*1
    return big_dic


In [83]:
#main 1
from elasticsearch import Elasticsearch
es = Elasticsearch('10.120.30.17:9200')

file_path='C:\\Users\\BigData\\Desktop\\groupnum.txt'
group_dic=put_id_group_in_dic(file_path)
id_keywords_file_path='C:\\Users\\BigData\\Desktop\\total_words.txt'
source_dic=merge_keywords(id_keywords_file_path, group_dic)
big_dic=source_to_big_dic(source_dic)

count=0
for gn in big_dic: 
    es_dic={"group":"","_id":[],"keywords":[],"count":[]}
    es_dic["group"]=gn
    for key in big_dic[gn]:
        if type(big_dic[gn][key]) != type([]):
            es_dic["keywords"].append(key)
            es_dic["count"].append(big_dic[gn][key])
        else:
            es_dic["_id"].append(big_dic[gn][key])
    new=int(gn)                
    res = es.index(index="test160", doc_type='test', id=new, body=es_dic)
    print count

step 1 finish
step 2 finish
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
step 3 finish =
1
2
3
4
5
